In [1]:
class CanNotOrderAdjacentElements(Exception):
    pass

def is_consec(ordering, subset):
    if len(subset)==0:
        return True
    
    indexes = list(map(ordering.index, subset))
    diff = max(indexes)-min(indexes)
    return diff==len(subset)-1
    
def check_order(connecteds, ordering):
    return all([is_consec(ordering,cc) for cc in connecteds])

def get_order(connecteds):
    from itertools import permutations
    from functools import reduce
    alphabet = reduce(set.union, connecteds, set())
    for order in permutations(alphabet):
        if check_order(connecteds,order):
            return order
    #Otherwise:
    raise(CanNotOrderAdjacentElementsError())
    

In [14]:
#CONFIG
import pytz
LOCAL_TIMEZONE = pytz.timezone("Australia/Perth")
SLOT_LENGTH_MINUTES = 30


In [5]:
def get_slots(start,end, exclusive_end=True, slot_length_minutes = SLOT_LENGTH_MINUTES):
    from dateutil import rrule
    until = end-timedelta(0,0,1) if exclusive_end else end
    slots = rrule.rrule(rrule.MINUTELY, 
                          interval = slot_length_minutes,
                          dtstart = start,
                          until = until,
                          )
    return list(slots)
    

In [22]:
def parse_datetime(datetime_str):
    import dateutil.parser
    
    raw = dateutil.parser.parse(datetime_str)
    return raw.astimezone(LOCAL_TIMEZONE)

def normalise(string): 
    import unicodedata
    import pylatex.utils

    string = str(pylatex.utils.escape_latex(string))
    return string.strip()
    

def load_list(csl):
    ldlst =  str.split(normalise(csl), ', ')
    if ldlst==['']:
        return []
    else:
        return ldlst

class session (object):
    def __init__(self, id,
                 start_time_str,end_time_str,
                 title,tags_str,people_str,
                 venues_str,description):
        self.id = int(id)
        self.start = parse_datetime(start_time_str)
        self.end = parse_datetime(end_time_str)
        self.title = normalise(title)
        self.tags = load_list(tags_str)
        self.people = load_list(people_str)
        self.venues = load_list(venues_str)
        self.description = normalise(description)
    
    @property
    def day(self):
        week_days   = ['Sunday', 
          'Monday', 
          'Tuesday', 
          'Wednesday', 
          'Thursday',  
          'Friday', 
          'Saturday']
        daynum = self.start.weekday()
        return week_days[daynum]
    
    
    @property
    def start_time(self):
        return self.start.strftime("%H:%M")
    
    @property
    def end_time(self):
        return self.end.strftime("%H:%M")
    
    @property
    def timeslots(self, slot_length_minutes=SLOT_LENGTH_MINUTES):
        return get_slots(self.start,self.end, slot_length_minutes)
        
            

In [16]:
import csv

sessions = []
with open('con_data/swancon_2014.csv', 'rU') as csvfile:
    con_csv = csv.reader(csvfile)
    next(con_csv) #Skip heading
    sessions = [session(*row) for row in con_csv]

sessions.sort(key=lambda ss: ss.start)



In [17]:
from collections import defaultdict
from defaultordereddict import DefaultOrderedDict
dtv_bookings = DefaultOrderedDict(lambda : defaultdict(lambda : defaultdict(str)))

import random

for session in sessions:
    for venue in session.venues:
        colorstring = "\cellcolor[gray]{%f} " % (0.5+0.5*random.random()) #HACK
        dtv_bookings[session.day][session.timeslots[0]][venue] = colorstring + session.title
        for slot in session.timeslots[1:]:
            dtv_bookings[session.day][slot][venue] =colorstring + "...cont..."


In [18]:
from pylatex.base_classes import BaseLaTeXNamedContainer

class Landscape(BaseLaTeXNamedContainer):
    def __init__(self, **kwargs):
        BaseLaTeXNamedContainer.__init__(self, 'landscape', packages=[Package('pdflscape')],**kwargs)
        
    

class Multicols(BaseLaTeXNamedContainer):
    def __init__(self,number_of_columns=2, **kwargs):
        argument=str(number_of_columns)
        BaseLaTeXNamedContainer.__init__(self, 'multicols', argument=argument, packages=[Package('multicol')],**kwargs)
        

In [19]:
class SessionOutOfSlotAlignmentError(Exception):
    def __init__(self, slot, day):
        self.slot=slot
        self.day=day
        

In [ ]:
import pylatex
from pylatex import Document, Section, Subsection, Subsubsection, Table, Package
from pylatex.command import Command
from lists import Description


venue_columns = list(get_order([session.venues for session in sessions]))
slots = None
doc = Document(documentclass="scrreprt")
doc.packages.append(Package('geometry', options=['tmargin=1cm','bmargin=1cm',
                                                 'lmargin=1cm','rmargin=1cm',]))
doc.packages.append(Package('xcolor', options=['table']))

with doc.create(Landscape()):
    with doc.create(Section('Timetable',numbering=False)):
        for day in dtv_bookings.keys():
            if len(dtv_bookings[day])>0:
                with doc.create(Subsection(day)):
                    table_spec = "c||"+"p{25mm}|"*len(venue_columns)
                    doc.append(Command('scriptsize'))
                    with doc.create(Table(table_spec)) as timetable:
                        timetable.add_row(["Time"]+venue_columns)
                        timetable.add_hline()
                        timetable.add_hline()

                        used_slots = set(dict.keys(dtv_bookings[day]))
                        slots = get_slots(min(used_slots),max(used_slots), exclusive_end=False)

                        for used_slot in used_slots:
                            if not used_slot in slots:
                                raise SessionOutOfSlotAlignmentError(used_slot,day)

                        for slot in slots:
                            venue_bookings = dtv_bookings[day][slot]
                            venue_slots = [venue_bookings[vv] for vv in venue_columns]

                            slot_time_str = slot.strftime("%H:%M")
                            timetable.add_row([slot_time_str] + venue_slots)

from itertools import groupby
with doc.create(Section('Program',numbering=False)):
    with doc.create(Multicols(2)):
        for day_name, day_session in groupby(sessions, lambda ss: ss.day):
            with doc.create(Subsection(day_name,numbering=False)):
                for session in day_session:
                    with doc.create(Subsubsection(session.title,numbering=False)):
                        with doc.create(Description()) as desc:
                            doc.append('\setlength{\itemsep}{0pt}')
                            desc.additem("Time:", session.start_time+" -- "+session.end_time)

                            if len(session.venues)>0:
                                desc.additem("Venue:", ", ".join(session.venues))
                            if len(session.people)>0:
                                desc.additem("People:", ", ".join(session.people))
                            if len(session.tags)>0:
                                desc.additem("Tags:", ", ".join(session.tags))
                        doc.append(session.description)



In [ ]:
from IPython.display import FileLink, FileLinks
with open("out/temp.tex", 'w') as temp_out:
    doc.dump(temp_out)

!lualatex --output-directory=out temp.tex
FileLinks("./out")

In [ ]:
FileLinks("./out")